In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month,date_sub,current_date,max,min
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)

# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)


In [ ]:
## TABLA SALES
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/gold/Completo_ventas/Ds_G_sales.parquet'
sales = spark.read.parquet(inpath)

# Maestra BANK_ID_TC
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Bank_ID/TC/Ds_S_Bank_ID_TC.parquet'
Bank_ID_TC = spark.read.parquet(inpath)

# print(sales.count())
# print(Bank_ID.count())

# Maestra BANK_ID_PSE
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Bank_ID/PSE/Ds_S_Bank_ID_PSE.parquet'
Bank_ID_PSE = spark.read.parquet(inpath)

In [ ]:
# tmp = Bank_ID.filter(col('IDENTIFIER') == '1638875939')
# display(tmp)

In [ ]:
# MAESTRA TARJETA CREDITO
sales_joined = sales.join(Bank_ID_TC,\
    sales["FILE_NUMBER"] == Bank_ID_TC["Expediente"],\
    "left_outer").\
    drop("Expediente")
#display(Bank_ID_TC.limit(10))

tabla_TC = sales_joined.select('PRODUCT','TEMA_MERCADEO','SUB_TEMA_MERCADEO','CardType','BinIssuer','NET_SOLD_AMT_ITX').\
    filter((year('REFERENCE_DATE') >= 2023) &
           (col('T_OPERATION_TYPE').isin('SALE','REFUND')) &
           (col('T_OPERATION_KIND').isin('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')) &
           (~col('PRODUCT_CODE').isin('CAN_INS'))).\
    groupBy('PRODUCT','TEMA_MERCADEO','SUB_TEMA_MERCADEO','CardType','BinIssuer').\
        agg(sum('NET_SOLD_AMT_ITX').alias('Amount')).\
    withColumn("Amount", format_number(col("Amount"), 0))

print(tabla_TC.count())

In [ ]:
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Financiera/CyberSource/"

write_table = tabla_TC.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.csv(inpath, header=True, mode="overwrite", sep=';')
 
#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]
 
# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Requerimiento_Cybersource.csv")

In [ ]:
# MAESTRA PSE
sales_joined = sales.join(Bank_ID_PSE,\
    sales["FILE_NUMBER"] == Bank_ID_PSE["Expediente"],\
    "left_outer").\
    drop("Expediente")
#display(Bank_ID_PSE.limit(10))

tabla_PSE = sales_joined.select('PRODUCT','TEMA_MERCADEO','SUB_TEMA_MERCADEO','Banco_Originador','NET_SOLD_AMT_ITX').\
    filter((year('REFERENCE_DATE') >= 2023) &
           (col('T_OPERATION_TYPE').isin('SALE','REFUND')) &
           (col('T_OPERATION_KIND').isin('SINGLE_ENTRY','PRODUCT_COMPOSITION','SIMPLE_PRODUCT')) &
           (~col('PRODUCT_CODE').isin('CAN_INS'))).\
    groupBy('PRODUCT','TEMA_MERCADEO','SUB_TEMA_MERCADEO','Banco_Originador').\
        agg(sum('NET_SOLD_AMT_ITX').alias('Amount')).\
    withColumn("Amount", format_number(col("Amount"), 0))

print(tabla_PSE.count())

26608


In [ ]:
#display(tabla.limit(10))

In [ ]:

inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Financiera/PSE/"

write_table = tabla_PSE.coalesce(1) # coalesce indica que la tabla se va a entregar en una sola particion
write_table.write.csv(inpath, header=True, mode="overwrite", sep=';')
 
#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]
 
# EN la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Requerimiento_PSE.csv")

True